In [ ]:
from orix.projections import StereographicProjection
from matplotlib import pyplot as plt

def get_ipf_color(vectors):
    """Map vectors to a color"""
    # the columns of this matrix should map to red, green blue respectively
    color_corners = np.array([[0, 1/np.sqrt(2), 1/np.sqrt(3)],
                              [0, 0, 1/np.sqrt(3)],
                              [1, 1/np.sqrt(2), 1/np.sqrt(3)]])
    color_mapper = np.linalg.inv(color_corners)

    # a bit of wrangling
    data_sol = vectors
    flattened = data_sol.reshape(-1, 3).T
    rgb_mapped = np.dot(color_mapper, flattened)
    print(rgb_mapped.shape)
    rgb_mapped = np.abs(rgb_mapped / rgb_mapped.max(axis=0)).T
    rgb_mapped = rgb_mapped.reshape(data_sol.shape)
    return rgb_mapped

get_ipf_color(v)

# proj.vector2xy(v)

proj = StereographicProjection()
x, y = proj.vector2xy(Vector3d(v))

c = get_ipf_color(v[:, [1,0,2]])

fig, ax = plt.subplots()
ax.scatter(x, y, c=c.clip(0, 1))


In [ ]:
vectors = np.random.rand(1000, 3)
vectors = (vectors.T / np.linalg.norm(vectors, axis=-1)).T

fig, ax = plt.subplots()

colors = get_ipf_color(vectors)
ax.scatter(vectors[:,0], vectors[:,1], c= / np.linalg.norm)



In [ ]:
from KED.utils import fibonacci_sphere, get_orientations_standard_triangle

pts = get_orientations_standard_triangle()
v = pts.apply((0,0,1))

c = get_ipf_color(v)

fig, ax = plt.subplots(ncols=2)
ax[0].scatter(v[:,0], v[:,1], marker='.', c=c)
ax[0].set_aspect('equal')

from scipy.spatial import ConvexHull, Voronoi, voronoi_plot_2d, cKDTree

xy = v[:, :-1]

vor = Voronoi(xy)

voronoi_plot_2d(vor, ax=ax[1], show_vertices=False, show_points=False)
ax[1].set_aspect('equal')

In [ ]:
from matplotlib.path import Path as mplPath

tree = cKDTree(xy)
dist, index = tree.query(xy, k=5)

i = 101
index = index[:, 1:]

fig, ax = plt.subplots()

ax.plot(*xy[i], marker='.')

xy1 = xy[index[i]]
ax.plot(*xy1.T, marker='.', color='k', ls='None')

verts = []
colors = []
for i, _xy in enumerate(xy):
    neighbours = xy[index[i]]
    # sort by angle
    neighbours = neighbours[np.arctan2(*(neighbours - _xy).T[::-1]).argsort()]
    
    if mplPath(neighbours).contains_point(_xy):
        continue

    verts.append(neighbours)
    colors.append(c[i])
    
    



In [ ]:
# create square grid with extents or standard triangle
grid = np.meshgrid(np.arange(0, v[:,0].max()), np.arange(0, v[:,1].max()))
pts = np.column_stack(tuple(g.ravel() for g in grid))

verts = np.stack(((0,0,1), (1, 0, 1), (1, 1, 1)))

for i, v in enumerate(verts):
    _cross = np.cross(verts[(i + 1) % len(verts)], v)
    temp = np.dot(pts, _cross)

    _cross_val = np.dot(center, _cross)
    if _cross_val > 0:
        pm = 1
    elif _cross_val < 0:
        pm = -1
    else:
        raise ValueError(f"Cannot work out direction.")

    # filter points
    pts = pts[pm * temp >= 0]

In [ ]:
from scipy.spatial.transform import Rotation

## generate grid
v1 = (1, 0, 1)
v2 = (1, 1, 1)

dtype = np.float32

num = 1000

if isinstance(v1, Rotation) and isinstance(v2, Rotation):
    # need angles
    a1 = v1.magnitude()
    a2 = v2.magnitude()
    # and unit vectors
    r1 = v1.as_rotvec() / a1
    r2 = v2.as_rotvec() / a2
else:
    z = np.array((0, 0, 1), dtype=dtype)
    v1 = np.asarray(v1, dtype=dtype)
    v2 = np.asarray(v2, dtype=dtype)

    # normalise vectors
    z /= np.linalg.norm(z)
    v1 /= np.linalg.norm(v1)
    v2 /= np.linalg.norm(v2)

    # find the rotation vector that transform z to v1, and v1 to v2
    r1 = np.cross(z, v1)
    r2 = np.cross(v1, v2)
    # normalise these vectors too
    r1 /= np.linalg.norm(r1)
    r2 /= np.linalg.norm(r2)

    # work out angles between the two vectors
    a1 = np.arccos(np.dot(z, v1))
    a2 = np.arccos(np.dot(v1, v2))

triangle = False
# approximate side length of grid for n points
side = int(((2 if triangle else 1) * num) ** 0.5 + 1)
grid = np.meshgrid(
    np.linspace(0, a1, int(side)), np.linspace(0, a2, int(side)), indexing="xy"
)


# just keep lower traingle
if triangle:
    idx = np.tril_indices(side)
else:
    idx = np.ones_like(grid[0], dtype=bool)

rot1 = Rotation.from_rotvec(grid[0][idx][..., np.newaxis] * r1)
rot2 = Rotation.from_rotvec(grid[1][idx][..., np.newaxis] * (grid[0][idx][..., np.newaxis] / grid[0][idx].max()) * r2)

# combine the rotations, order matters
rot = rot2 * rot1

shape = (len(rot),) if triangle else (side, side)

In [ ]:
# have some orientations (rot, scipy in this case) from a grid (of size side x side) that represent the standard triangle
xyz = rot.apply((0, 0, 1)).reshape(side, side, 3)
# split into components
x, y, z = xyz[..., 0], xyz[..., 1], xyz[..., 2]

# get grid center points for colour evaluation
x_mid = np.diff(np.diff(x, axis=0), axis=1) + x[:-1, :-1]
y_mid = np.diff(np.diff(y, axis=0), axis=1) + y[:-1, :-1]
z_mid = np.diff(np.diff(z, axis=0), axis=1) + z[:-1, :-1]
xyz_mid = np.column_stack((x_mid.ravel(), y_mid.ravel(), z_mid.ravel()))

c = get_ipf_color(xyz_mid).reshape(*x_mid.shape, 3)

fig, ax = plt.subplots()
# project and show colours
ax.pcolorfast(x / (z+1), y / (z+1), c.clip(0, 1))

In [ ]:
fig, ax = plt.subplots()
ax.imshow(get_ipf_color(xyz).reshape(side, side, -1))

In [ ]:
from diffsims.generators.rotation_list_generators import get_beam_directions_grid, get_fundamental_zone_grid, get_list_from_orix, get_icosahedral_mesh_vertices,beam_directions_grid_to_euler
from orix.quaternion.symmetry import Oh
from scipy.spatial.transform import Rotation
from matplotlib import pyplot as plt

rot = get_beam_directions_grid('cubic', 3)

v = Rotation.from_euler('ZXZ', rot, degrees=True).apply((0,0,1))

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(v[:, 0], v[:, 1], v[:, 2], '.')

In [ ]:
from orix.quaternion.symmetry import Oh, C1

rot = get_fundamental_zone_grid(2, space_group=225)
v = Rotation.from_euler('ZXZ', rot, degrees=True).apply((0,0,1))

from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(v[:, 0], v[:, 1], v[:, 2], '.')

In [ ]:
from orix.quaternion import Rotation
import numpy as np
from orix.quaternion import get_point_group, OrientationRegion
from orix.vector import Vector3d

rot = beam_directions_grid_to_euler(get_icosahedral_mesh_vertices(1))

q = Rotation.from_euler(np.deg2rad(rot))
point_group = get_point_group(225, proper=True)
fundamental_region = OrientationRegion.from_symmetry(point_group)
rot = get_list_from_orix(q[q < fundamental_region], rounding=4)

v = (Rotation.from_euler(np.deg2rad(rot)) * Vector3d.zvector()).data

from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(v[:, 0], v[:, 1], v[:, 2], '.')

ax

In [ ]:

fig, ax = plt.subplots()

c = get_ipf_color(xyz)

x = xyz[:, 0]
y = xyz[:, 1]
z = xyz[:, 2]

ax.scatter(x / (1 + z), y / (1 + z), c=c)

ax.set_aspect('equal')
